In [84]:
## Import Dependencies
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
import tensorflow as tf
import keras_tuner as kt

In [85]:
## Ticker List, start date
tickers = ["BTC-USD", #Bitcoin (deleted - "ETH-USD", "LTC-USD", "BITW", ether, litecoin, top 10 crypto index fund)
           "GLD", "SLV", "CL=F", #Gold, silver, crude
           "VIXY", #VIX Short-Term Futures ETF
           "^IXIC", "^GSPC", "^DJI", #Nasdaq, s&p, dow
           "META", "AMZN", "AAPL", "NFLX", "GOOG", "TSLA", #faangs, telsa
           "JPM", "WFC", "C", "BAC", #Big US Banks JPM, WFC, C, BAC
           "UUP", #usd bull fund
           "IEF" #iShares 7-10 Year Treasury Bond 
           ]
start_date = "2013-12-31"
filter_date = "2014-11-06" #"2014-11-06" - first day BTC tracked on yfinance, "2018-04-16" - first day to include entire sentiment analysis

In [86]:
## Interval lengths, dates
long = 50
med = 15
short = 5
start_date = datetime.strptime(start_date, '%Y-%m-%d').date() - timedelta(days = long)
filter_date = datetime.strptime(filter_date, '%Y-%m-%d').date()

In [87]:
## Random state
seed = 15

In [88]:
## Quantiles, target return
quantiles = [.01, .05, .1, .5, .9, .95, .99]
model_q = .5
#target_return = 0

In [89]:
## Look up ticker and create csv
for ticker in tickers:
    data = pd.DataFrame(yf.download(ticker, start=start_date))
    filename = ticker + '.csv'
    data.to_csv(filename)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [90]:
## Create comprehensive list of dates
#start = datetime.strptime(start_date, '%Y-%m-%d').date() - timedelta(days = long)
start = start_date
end = date.today() - timedelta(days = 1)
delta = timedelta(days=1)

dates = []
while start <= end:
    dates.append(start.isoformat())
    start += delta

dates = pd.DataFrame(dates)
dates.columns =['Date']
dates['Date'] = pd.to_datetime(dates['Date'])
dates

,Date
0,2013-11-11
1,2013-11-12
2,2013-11-13
3,2013-11-14
4,2013-11-15
...,...
3435,2023-04-08
3436,2023-04-09
3437,2023-04-10
3438,2023-04-11


In [91]:
## Create empty pd df for classification analysis
df_class = pd.DataFrame(columns=['Ticker', 'avgvol_last_' + str(long), 'hi_to_lo_last_' + str(long), 'avgvol_last_' + str(med), 'hi_to_lo_last_' + str(med),
                                 'avgvol_last_' + str(short), 'hi_to_lo_last_' + str(short), 'avgvol_last_1', 'hi_to_lo_last_1'])

## Create empty pd df for quantile comparison
df_quant = pd.DataFrame(index=quantiles)

## Create empty pd df for logistic regression comparison
df_logist = pd.DataFrame(columns=['Ticker', 'percentile_' + str(model_q), 'Accuracy', 'TP_rate', 'TN_rate', 'Pos_Predicted_Value', 'Neg_Predicted_Value'])
df_logist_profit = pd.DataFrame(columns=['Ticker', 'target_return', 'Accuracy', 'TP_rate', 'TN_rate', 'Pos_Predicted_Value', 'Neg_Predicted_Value'])

## Loop thru asset files and add metrics
for ticker in tickers:
    ## Upload csv datasets
    filename = ticker + '.csv'
    temp = pd.read_csv(filename)
    
    ## Format date
    temp['Date'] = pd.to_datetime(temp['Date'])
    
    ## Merge dfs to capture all dates including weekends and market holidays
    df = pd.merge(dates, temp, how = 'left', on='Date')
    
    ## Fill in weekends and market holidays using last trading day
    df['Open'].fillna(method='ffill', inplace=True)
    df['High'].fillna(method='ffill', inplace=True)
    df['Low'].fillna(method='ffill', inplace=True)
    df['Close'].fillna(method='ffill', inplace=True)
    df['Adj Close'].fillna(method='ffill', inplace=True)
    df['Volume'].fillna(method='ffill', inplace=True)

    ## Shift metrics to get yesterday's value
    df['hi_shift'] = df['High'].shift(1)
    df['lo_shift'] = df['Low'].shift(1)
    df['adjclose_shift'] = df['Adj Close'].shift(1)
    df['vol_shift'] = df['Volume'].shift(1)
    
    ## Calculate 'Adj Close' for past and future
    df['close_last_long'] = df['Adj Close'].shift(long)
    df['close_last_med'] = df['Adj Close'].shift(med)
    df['close_last_short'] = df['Adj Close'].shift(short)
    df['close_last_1'] = df['Adj Close'].shift(2)
    df['close_next_1'] = df['Adj Close']
    df['close_next_short'] = df['Adj Close'].shift(-short + 1)
    df['close_next_med'] = df['Adj Close'].shift(-med + 1)
    df['close_next_long'] = df['Adj Close'].shift(-long + 1)
    
    ## Calculate short, med, long hi/lo, avg vol, return
    ## Past
    df['avgvol_last_long'] = df['vol_shift'].rolling(long).sum() / long
    df['vol_hi_lo_last_long'] = (df['vol_shift'].rolling(long).max() / df['vol_shift'].rolling(long).min()) - 1
    df['hi_to_lo_last_long'] = (df['adjclose_shift'].rolling(long).max() / df['adjclose_shift'].rolling(long).min()) - 1
    df['return_last_long'] = (df['adjclose_shift'] / df['close_last_long']) - 1
    df['avgvol_last_med'] = df['vol_shift'].rolling(med).sum() / med
    df['vol_hi_lo_last_med'] = (df['vol_shift'].rolling(med).max() / df['vol_shift'].rolling(med).min()) - 1
    df['hi_to_lo_last_med'] = (df['adjclose_shift'].rolling(med).max() / df['adjclose_shift'].rolling(med).min()) - 1
    df['return_last_med'] = (df['adjclose_shift'] / df['close_last_med']) - 1
    df['avgvol_last_short'] = df['vol_shift'].rolling(short).sum() / short
    df['vol_hi_lo_last_short'] = (df['vol_shift'].rolling(short).max() / df['vol_shift'].rolling(short).min()) - 1
    df['hi_to_lo_last_short'] = (df['adjclose_shift'].rolling(short).max() / df['adjclose_shift'].rolling(short).min()) - 1
    df['return_last_short'] = (df['adjclose_shift'] / df['close_last_short']) - 1
    df['hi_to_lo_last_1'] = (df['hi_shift'] / df['lo_shift']) - 1
    df['return_last_1'] = (df['adjclose_shift'] / df['close_last_1']) - 1
    ## Future
    df['return_next_1'] = (df['close_next_1'] / df['adjclose_shift']) - 1
    df['return_next_short'] = (df['close_next_short'] / df['adjclose_shift']) - 1
    df['return_next_med'] = (df['close_next_med'] / df['adjclose_shift']) - 1
    df['return_next_long'] = (df['close_next_long'] / df['adjclose_shift']) - 1
    
    ## Filter out Nulls
    df = df.dropna()

    ## Find last date in dfs
    max_date = df['Date'].max()
    
    ## Drop unused fields
    df = df.drop(columns=['hi_shift', 'lo_shift', 'adjclose_shift',
                          'close_last_long', 'close_last_med', 'close_last_short', 'close_last_1', 'close_next_1', 'close_next_long', 'close_next_med', 'close_next_short'])
    
    ## Rename, add ticker, and rearrange columns
    ## Rename
    df = df.rename(columns={"vol_shift": "avgvol_last_1",
                            "avgvol_last_long": "avgvol_last_" + str(long), 
                            "vol_hi_lo_last_long": "vol_hi_lo_last_" + str(long),
                            "hi_to_lo_last_long": "hi_to_lo_last_" + str(long),
                            "return_last_long": "return_last_" + str(long),
                            "avgvol_last_med": "avgvol_last_" + str(med), 
                            "vol_hi_lo_last_med": "vol_hi_lo_last_" + str(med),
                            "hi_to_lo_last_med": "hi_to_lo_last_" + str(med),
                            "return_last_med": "return_last_" + str(med),
                            "avgvol_last_short": "avgvol_last_" + str(short), 
                            "vol_hi_lo_last_short": "vol_hi_lo_last_" + str(short),
                            "hi_to_lo_last_short": "hi_to_lo_last_" + str(short),
                            "return_last_short": "return_last_" + str(short),
                            "return_next_long": "return_next_" + str(long),
                            "return_next_med": "return_next_" + str(med),
                            "return_next_short": "return_next_" + str(short)})
    ## Add ticker
    df['Ticker'] = ticker

    ## Rearrange
    df = df[['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 
             'avgvol_last_' + str(long), 'vol_hi_lo_last_' + str(long), 'hi_to_lo_last_' + str(long), 'return_last_' + str(long),
             'avgvol_last_' + str(med), 'vol_hi_lo_last_' + str(med), 'hi_to_lo_last_' + str(med), 'return_last_' + str(med),
             'avgvol_last_' + str(short), 'vol_hi_lo_last_' + str(short), 'hi_to_lo_last_' + str(short), 'return_last_' + str(short),
             'avgvol_last_1', 'hi_to_lo_last_1', 'return_last_1',
             'return_next_1', 'return_next_' + str(short), 'return_next_' + str(med), 'return_next_' + str(long)]]
    
    ## Filter dates
    df = df.loc[df['Date'] >= pd.to_datetime(filter_date)]

    ## Write to individual ticker csv
    df.to_csv(filename)

    ###########################################

    ## df_class
    ## Filter df for max_date
    df_temp = df.loc[df['Date'] == max_date]
    df_temp = df_temp[['Ticker', 'avgvol_last_' + str(long), 'vol_hi_lo_last_' + str(long), 'hi_to_lo_last_' + str(long), 
                                 'avgvol_last_' + str(med), 'vol_hi_lo_last_' + str(med), 'hi_to_lo_last_' + str(med),
                                 'avgvol_last_' + str(short), 'vol_hi_lo_last_' + str(short),  'hi_to_lo_last_' + str(short), 
                                 'avgvol_last_1', 'hi_to_lo_last_1']]

    ## Add row to df_class
    df_class = df_class.append(df_temp, ignore_index=True)

    ###########################################
    
    ## df_quant
    ## Create df for quantiles of each ticker
    df_quant_temp = df[['return_next_' + str(long)]]
    df_quant_temp = df_quant_temp.quantile(quantiles)
    df_quant_temp = df_quant_temp.rename(columns={'return_next_' + str(long): ticker})

    # Merge df with df_blank
    df_quant = pd.merge(df_quant, df_quant_temp, left_index=True, right_index=True)

    ###########################################

    ## df_logist
    ## Get the quantile to use for binary classification
    q_model = df_quant.filter(items=[model_q], axis=0).iloc[0][ticker]

    ## Add 'Outcome' field 
    df['Outcome'] = np.where(df['return_next_' + str(long)] >= q_model, 1, 0)

    ## Establish y and X variables
    y = df['Outcome']
    X = df.drop(columns=['Outcome', 'Date', 'Ticker', 
                         'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
                         'return_next_1', 'return_next_' + str(short), 'return_next_' + str(med)])

    ## Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, stratify=y)

    ## Create Logistic Regression Model
    classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=seed)

    ## Fit model on training data
    classifier.fit(X_train, y_train)

    ## Make predictions
    predictions = classifier.predict(X_test)
    
    ## Calculate confusion matrix, accuracy score, TP_rate, TN_rate
    cm = confusion_matrix(y_test, predictions)
    accuracy = accuracy_score(y_test, predictions)
    TP = cm[1][1]
    FN = cm[1][0]
    FP = cm[0][1]
    TN = cm[0][0]
    TP_rate = TP / (TP + FN)
    TN_rate = TN / (TN + FP)
    PPV_rate = TP / (TP + FP)
    NPV_rate = TN / (TN + FN)

    ## Append df_logist with each asset score
    df_logist_temp = {'Ticker':ticker, 'percentile_' + str(model_q): q_model, 'Accuracy':accuracy, 'TP_rate': TP_rate, 'TN_rate': TN_rate, 'Pos_Predicted_Value': PPV_rate, 'Neg_Predicted_Value': NPV_rate}
    df_logist = df_logist.append(df_logist_temp, ignore_index=True)

    ###########################################
    
    # ## df_logist_prof
    # ## Add 'Outcome' field 
    # df['Outcome2'] = np.where(df['return_next_' + str(long)] >= target_return, 1, 0)

    # ## Establish y and X variables
    # y2 = df['Outcome2']
    # X2 = df.drop(columns=['Outcome', 'Outcome2', 'Date', 'Ticker', 
    #                      'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
    #                      'return_next_1', 'return_next_' + str(short), 'return_next_' + str(med)])

    # ## Split into train and test
    # X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=seed, stratify=y2)

    # ## Create Logistic Regression Model
    # classifier2 = LogisticRegression(solver='lbfgs', max_iter=200, random_state=seed)

    # ## Fit model on training data
    # classifier2.fit(X_train2, y_train2)

    # ## Make predictions
    # predictions2 = classifier2.predict(X_test2)
    
    # ## Calculate confusion matrix, accuracy score, TP_rate, TN_rate
    # cm2 = confusion_matrix(y_test2, predictions2)
    # accuracy2 = accuracy_score(y_test2, predictions2)
    # TP2 = cm2[0][0]
    # FN2 = cm2[0][1]
    # FP2 = cm2[1][0]
    # TN2 = cm2[1][1]
    # TP_rate2 = TP2 / (TP2 + FN2)
    # TN_rate2 = TN2 / (TN2 + FP2)
    # PPV_rate2 = TP2 / (TP2 + FP2)
    # NPV_rate2 = TN2 / (TN2 + FN2)

    # ## Append df_logist with each asset score
    # df_logist_temp2 = {'Ticker':ticker, 'target_return': target_return, 'Accuracy':accuracy2, 'TP_rate': TP_rate2, 'TN_rate': TN_rate2, 'Pos_Predicted_Value': PPV_rate2, 'Neg_Predicted_Value': NPV_rate2}
    # df_logist_profit = df_logist_profit.append(df_logist_temp2, ignore_index=True)


###########################################

## Write to classification csv
df_class.to_csv('df_class.csv')

## Write to quantile csv
df_quant.to_csv('df_quant.csv')

## Write to logistic csv
df_logist.to_csv('df_logist.csv')

## Write to logistic profit csv
# df_logist_profit.to_csv('df_logist_profit.csv')

In [92]:
#########################################################################################################################

In [93]:
## K-means Classification (Elbow Method to select k)

## Create new variables for log of average volume
df_class['trans_avgvol_last_1'] = np.log2(df_class['avgvol_last_1'])
df_class['trans_avgvol_last_' + str(short)] = np.log2(df_class['avgvol_last_' + str(short)])
df_class['trans_avgvol_last_' + str(med)] = np.log2(df_class['avgvol_last_' + str(med)])
df_class['trans_avgvol_last_' + str(long)] = np.log2(df_class['avgvol_last_' + str(long)])

## Variable list
class_cols = ['trans_avgvol_last_' + str(long), 'vol_hi_lo_last_' + str(long), 'hi_to_lo_last_' + str(long), 
              'trans_avgvol_last_' + str(med), 'vol_hi_lo_last_' + str(med), 'hi_to_lo_last_' + str(med),
              'trans_avgvol_last_' + str(short), 'vol_hi_lo_last_' + str(short), 'hi_to_lo_last_' + str(short), 
              'trans_avgvol_last_1', 'hi_to_lo_last_1']

## Scale the data
df_class_scaled = StandardScaler().fit_transform(df_class[class_cols])

## Create a DataFrame with the scaled data
df_class_scaled = pd.DataFrame(df_class_scaled, columns= class_cols)

## Copy the tickers names from the original data
df_class_scaled["Ticker"] = df_class["Ticker"]

## Set the Ticker column as index
df_class_scaled = df_class_scaled.set_index("Ticker")

## Create a a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

## Create a for-loop where each value of k is evaluated using the K-means algorithm
for i in k:
    k_model = KMeans(n_clusters=i, random_state=seed)
    k_model.fit(df_class_scaled)
    inertia.append(k_model.inertia_)

## Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

## Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k)

c:\Users\raminskim\Anaconda3\envs\PythonData2\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

In [94]:
## K-means Classification (Plot the clusters)

## Initialize the K-Means model with n_clusters
model = KMeans(n_clusters=4, random_state=seed)

## Fit the model for the df_stocks_scaled DataFrame
model.fit(df_class_scaled)

## Predict the model segments (clusters)
asset_clusters = model.predict(df_class_scaled)

## Create a new column in the DataFrame with the predicted clusters
df_class_scaled["Assigned_Cluster"] = asset_clusters

## Create a scatter plot with x="AnnualVariance:,  y="AnnualReturn"
df_class_scaled.hvplot.scatter(
    x='trans_avgvol_last_' + str(long),
    y='hi_to_lo_last_' + str(long),
    by="Assigned_Cluster",
    hover_cols = ["Ticker"], 
    title = "Scatter Plot by Asset")

:NdOverlay   [Assigned_Cluster]
   :Scatter   [trans_avgvol_last_50]   (hi_to_lo_last_50,Ticker)

In [95]:
#########################################################################################################################

In [96]:
df_class_scaled.head()

,trans_avgvol_last_50,vol_hi_lo_last_50,hi_to_lo_last_50,trans_avgvol_last_15,vol_hi_lo_last_15,hi_to_lo_last_15,trans_avgvol_last_5,vol_hi_lo_last_5,hi_to_lo_last_5,trans_avgvol_last_1,hi_to_lo_last_1,Assigned_Cluster
Ticker,,,,,,,,,,,,
BTC-USD,2.373883,0.381551,1.171366,2.412364,0.165551,1.784243,2.402365,0.810754,-0.142568,2.466345,1.077153,3
GLD,-0.749350,0.049803,-0.746565,-0.816733,-0.043754,-1.007193,-0.766108,-0.400929,-1.099534,-0.884758,-1.105599,1
SLV,-0.369338,-0.038101,-0.474110,-0.401836,-0.465494,-0.782096,-0.358788,-0.183827,-1.142239,-0.504167,-0.849693,1
CL=F,-1.906766,1.108655,-0.496616,-1.872526,1.765707,-0.333907,-1.998015,3.730768,-1.178281,-1.747609,0.089696,0
VIXY,-0.742796,-0.376966,0.240256,-0.675266,-0.084608,1.606708,-0.565370,-0.270772,3.011215,-0.549687,1.843336,2


In [97]:
## PCA (Explore # components)

## Drop 'Assigned Cluster' column
df_class_scaled_pca = df_class_scaled.drop(columns = ['Assigned_Cluster'])

## Instantiate the PCA instance and declare the number of PCA variables
pca_n = len(df_class_scaled_pca.columns)
pca = PCA(n_components = pca_n)

## Fit the PCA model on the transformed df
df_class_pca = pca.fit_transform(df_class_scaled_pca)

## Create the PCA DataFrame
df_class_pca = pd.DataFrame(
    df_class_pca,
    columns= ['PCA'+str(i+1) for i in range(pca_n)]
    )

## Merge with tickers
df_class_pca = pd.merge(df_class['Ticker'], df_class_pca, left_index = True, right_index = True)

## Review the PCA DataFrame
df_class_pca

,Ticker,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,PCA11
0,BTC-USD,-4.158880,0.255354,3.495501,-0.204832,-0.027951,-0.370695,-0.332823,-0.016155,0.056387,-0.001553,0.034193
1,GLD,1.938475,-1.283414,-0.946589,0.149555,0.625643,-0.045593,0.193944,-0.105122,0.012064,-0.005813,0.031134
2,SLV,1.083283,-1.242576,-0.621331,-0.146789,0.766450,-0.273474,0.039364,0.056929,0.057250,-0.020571,-0.006792
3,CL=F,5.084254,1.233716,1.379823,-1.890977,-0.538785,-0.985202,0.252012,0.179299,-0.008534,0.010693,-0.011496
4,VIXY,-0.521735,3.383635,-1.397195,0.669531,-1.629648,-0.059550,-0.414530,-0.024359,0.038647,-0.042010,-0.006560
5,^IXIC,-3.451551,-1.951183,0.255461,-0.212899,-0.437308,0.017998,0.052443,0.173714,-0.015808,0.003103,-0.016190
6,^GSPC,-3.053829,-2.387146,0.213665,-0.234979,-0.441821,-0.005907,-0.080726,0.048820,-0.006992,-0.003419,-0.023866
7,^DJI,-1.499007,-1.754107,-0.666198,-0.415521,-0.470422,-0.191134,-0.092388,-0.136548,-0.011065,0.001329,-0.015425
8,META,-0.035086,1.357402,0.945851,0.144531,1.933138,-0.319000,-0.470883,-0.184487,-0.066956,-0.027518,-0.019132
9,AMZN,-0.608324,-0.055328,-0.200439,0.525010,0.098819,0.168772,0.881477,0.209169,-0.047458,-0.028314,0.038912


In [98]:
## Calculate the PCA explained variance ratio
list(pca.explained_variance_ratio_)

[0.4952800710037289,
 0.21443926509508146,
 0.14594735794474878,
 0.056641845450242166,
 0.046901472088943505,
 0.02509133372618946,
 0.012993019644903087,
 0.0024118045921438357,
 0.00019274199050060362,
 6.50420871129647e-05,
 3.604637640523534e-05]

In [99]:
## PCA (run with selected # components)

## Select pca # 
pca_n = 3

## Instantiate the PCA instance and declare the number of PCA variables
pca = PCA(n_components = pca_n)

## Fit the PCA model on the transformed df
df_class_pca = pca.fit_transform(df_class_scaled_pca)

## Create the PCA DataFrame
df_class_pca = pd.DataFrame(
    df_class_pca,
    columns=['PCA'+str(i+1) for i in range(pca_n)])

## Merge with tickers
df_class_pca = pd.merge(df_class['Ticker'], df_class_pca, left_index = True, right_index = True)

## Review the PCA DataFrame
df_class_pca

,Ticker,PCA1,PCA2,PCA3
0,BTC-USD,-4.158880,0.255354,3.495501
1,GLD,1.938475,-1.283414,-0.946589
2,SLV,1.083283,-1.242576,-0.621331
3,CL=F,5.084254,1.233716,1.379823
4,VIXY,-0.521735,3.383635,-1.397195
5,^IXIC,-3.451551,-1.951183,0.255461
6,^GSPC,-3.053829,-2.387146,0.213665
7,^DJI,-1.499007,-1.754107,-0.666198
8,META,-0.035086,1.357402,0.945851
9,AMZN,-0.608324,-0.055328,-0.200439


In [100]:
## PCA (use elbow method to select k)

## make copy of df for fitting
df_class_pca_fit = df_class_pca.copy()
df_class_pca_fit = df_class_pca_fit.drop(columns=['Ticker'])

## Create a a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

## Append the value of the computed inertia from the `inertia_` attribute of teh KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=seed)
    k_model.fit(df_class_pca_fit)
    inertia.append(k_model.inertia_)

## Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

## Plot the Elbow Curve
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k)

c:\Users\raminskim\Anaconda3\envs\PythonData2\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

In [101]:
## PCA (use elbow method to select k)

## Define the model with 4 clusters
model = KMeans(n_clusters=4, random_state=seed)

## Fit the model
model.fit(df_class_pca_fit)

## Make predictions
k_4 = model.predict(df_class_pca_fit)

## Create a copy of the PCA DataFrame
df_class_pca_predictions = df_class_pca.copy()

## Add a class column with the labels
df_class_pca_predictions["Segments"] = k_4

## Plot the clusters
df_class_pca_predictions.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="Segments",
    hover_cols = ["Ticker"])
#* \
#df_class_pca_predictions.hvplot.labels(x="PCA1", y="PCA2", text="Ticker", text_baseline='top', by="Segments", hover=False).opts(fontsize={'xticks': 8, 'labels': 10})

:NdOverlay   [Segments]
   :Scatter   [PCA1]   (PCA2,Ticker)

In [102]:
## PCA look at most important features for each PC
# get the index of the most important feature on EACH component
# LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(pca_n)]
initial_feature_names = df_class_scaled.columns

# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(pca_n)]

# LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(pca_n)}

# # build the dataframe
df_pca_importance = pd.DataFrame(dic.items())
#df_pca_importance = df_pca_importance.rename(columns= {0: "PC#"}, inplace = True)
df_pca_importance

,0,1
0,PC1,trans_avgvol_last_5
1,PC2,hi_to_lo_last_1
2,PC3,vol_hi_lo_last_15


In [103]:
#########################################################################################################################

In [104]:
## Additional features data sources
# Consumer Sentiment Data
# http://www.sca.isr.umich.edu/tables.html

# Gallup polls instituional confidence 
# https://news.gallup.com/poll/1597/confidence-institutions.aspx 

# CPI Data
# https://www.bls.gov/regions/mid-atlantic/data/consumerpriceindexhistorical_us_table.htm

# US GDP
# https://www.macrotrends.net/countries/USA/united-states/gdp-gross-domestic-product


## Unused #############

# Pew polls instituional confidence
# https://www.pewresearch.org/politics/2022/06/06/public-trust-in-government-1958-2022/

# OECD trust in government
# https://data.oecd.org/gga/trust-in-government.htm

In [105]:
## Date / Time
dates2 = dates.copy()
dates2['Day'] = pd.to_datetime(dates2['Date']).dt.dayofweek
dates2['Month'] = pd.to_datetime(dates2['Date']).dt.month
dates2.loc[dates2['Month'] >= 10, 'Quarter'] = 4
dates2.loc[dates2['Month'] <= 9, 'Quarter'] = 3
dates2.loc[dates2['Month'] <= 6, 'Quarter'] = 2
dates2.loc[dates2['Month'] <= 3, 'Quarter'] = 1
dates2

,Date,Day,Month,Quarter
0,2013-11-11,0,11,4.0
1,2013-11-12,1,11,4.0
2,2013-11-13,2,11,4.0
3,2013-11-14,3,11,4.0
4,2013-11-15,4,11,4.0
...,...,...,...,...
3435,2023-04-08,5,4,2.0
3436,2023-04-09,6,4,2.0
3437,2023-04-10,0,4,2.0
3438,2023-04-11,1,4,2.0


In [106]:
## Consumer Sentiment Data
# http://www.sca.isr.umich.edu/tables.html

## Load file
csi = pd.read_csv('Resources/tbmics.csv')

## Convert month to month number
d = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
csi['Month'] = csi['Month'].map(d)

## Create date field, using first day of the month
csi['Date'] = pd.to_datetime(dict(year=csi['YYYY'], month=csi['Month'], day=1))

## Exclude unnecessary columns
csi = csi[['Date', 'ICS_ALL']]

## Rename columns
csi = csi.rename(columns={"ICS_ALL": "CSI"})

## Merge with date list
csi = pd.merge(dates, csi, how = 'left', on='Date')

## Interpolate in between readings
csi['CSI'].interpolate(inplace=True)

In [107]:
## Gallup Polling Data
# https://news.gallup.com/poll/1597/confidence-institutions.aspx

## File list
polls = ['Resources/gallup_banks.xlsx', 'Resources/gallup_bigbiz.xlsx', 'Resources/gallup_police.xlsx', 'Resources/gallup_pres.xlsx']

## Date List
gallup = dates

for poll in polls:
    ## Load file and take first 30 rows
    gallup_temp = pd.read_excel(poll)
    gallup_temp = gallup_temp.iloc[:30]

    ## Create date field, using first day of the month
    gallup_temp['Date'] = pd.to_datetime(dict(year=gallup_temp['Year'], month=7, day=1))

    ## Exclude unnecessary columns
    gallup_temp = gallup_temp[['Date', 'Great deal/Quite a lot']]

    ## Rename columns
    gallup_temp = gallup_temp.rename(columns={"Great deal/Quite a lot": poll[10:len(poll)-5]})

    ## Merge with date list
    gallup_temp = pd.merge(dates, gallup_temp, how = 'left', on='Date')

    ## Interpolate in between readings
    gallup_temp[poll[10:len(poll)-5]].interpolate(inplace=True)

    ## Merge into gallup df
    gallup = pd.merge(gallup, gallup_temp, how = 'left', on='Date')

In [108]:
## CPI Data
# https://www.bls.gov/regions/mid-atlantic/data/consumerpriceindexhistorical_us_table.htm

## Load file
cpi = pd.read_excel('Resources/cpi.xlsx')

## Convert month to month number
d = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
cpi['Month'] = cpi['Month'].map(d)

## Create date field, using first day of the month
cpi['Date'] = pd.to_datetime(dict(year=cpi['Year'], month=cpi['Month'], day=1))

## Exclude unnecessary columns
cpi = cpi[['Date', 'CPI']]

## Merge with date list
cpi = pd.merge(dates, cpi, how = 'left', on='Date')

## Interpolate in between readings
cpi['CPI'].interpolate(inplace=True)

In [109]:
## US GDP
# https://www.macrotrends.net/countries/USA/united-states/gdp-gross-domestic-product

## Load file
gdp = pd.read_csv('Resources/united-states-gdp-gross-domestic-product.csv')

## Convert Date to proper date format
gdp['date'] = pd.to_datetime(gdp['date'])

## Exclude unnecessary columns
gdp = gdp[['date', ' Annual % Change']]

## Rename columns
gdp = gdp.rename(columns={"date": 'Date',
                          ' Annual % Change': 'GDP'})

## Merge with date list
gdp = pd.merge(dates, gdp, how = 'left', on='Date')

## Interpolate in between readings
gdp['GDP'].interpolate(inplace=True)

gdp

,Date,GDP
0,2013-11-11,NaN
1,2013-11-12,NaN
2,2013-11-13,NaN
3,2013-11-14,NaN
4,2013-11-15,NaN
...,...,...
3435,2023-04-08,5.9455
3436,2023-04-09,5.9455
3437,2023-04-10,5.9455
3438,2023-04-11,5.9455


In [110]:
## Sentiment Analysis

## Load file
vader = pd.read_csv('../JennS/Sentiments/all_crypto_sentiments.csv')

## Convert Date to proper date format
vader['Date'] = pd.to_datetime(vader['begins_at'])

## Rename columns
vader['vader_shift'] = vader['vader_prediction'].shift(1)
vader['sentiment_shift'] = vader['sentiment'].shift(1)

## Calculate rolling values
vader['vader_avg_last_long'] = vader['vader_shift'].rolling(long).sum() / long
vader['vader_avg_last_med'] = vader['vader_shift'].rolling(med).sum() / med
vader['vader_avg_last_short'] = vader['vader_shift'].rolling(short).sum() / short

## Rename columns
vader = vader.rename(columns={'vader_shift': 'vader_avg_last_1',
                              'vader_avg_last_long': 'vader_avg_last_' + str(long),
                              'vader_avg_last_med': 'vader_avg_last_' + str(med),
                              'vader_avg_last_short': 'vader_avg_last_' + str(short)})

## Exclude unnecessary columns
vader = vader[['Date', 'vader_avg_last_' + str(long), 'vader_avg_last_' + str(med), 'vader_avg_last_' + str(short), 'vader_avg_last_1']]

vader

,Date,vader_avg_last_50,vader_avg_last_15,vader_avg_last_5,vader_avg_last_1
0,2018-02-25,NaN,NaN,NaN,NaN
1,2018-02-26,NaN,NaN,NaN,0.010650
2,2018-02-27,NaN,NaN,NaN,0.055795
3,2018-02-28,NaN,NaN,NaN,-0.034722
4,2018-03-01,NaN,NaN,NaN,0.065192
...,...,...,...,...,...
1834,2023-03-05,0.068715,0.082024,0.098283,0.058089
1835,2023-03-06,0.068340,0.085956,0.094568,0.093306
1836,2023-03-07,0.067922,0.084390,0.080840,0.061426
1837,2023-03-08,0.067206,0.081315,0.062457,0.049406


In [111]:
## Additional data features df (merge all)
df_addl = pd.merge(dates, dates2, how='left', on='Date')
df_addl = pd.merge(df_addl, csi, how = 'left', on='Date')
df_addl = pd.merge(df_addl, gallup, how='left', on='Date')
df_addl = pd.merge(df_addl, cpi, how='left', on='Date')
df_addl = pd.merge(df_addl, gdp, how='left', on='Date')
#df_addl = pd.merge(df_addl, vader, how='left', on='Date')

## Filter dates
df_addl = df_addl.loc[df_addl['Date'] >= pd.to_datetime(filter_date)]

## Write to additional csv
df_addl.to_csv('df_addl.csv')

In [112]:
#########################################################################################################################

In [113]:
## MODEL 1 - Logistic regression - BTC (WITHOUT additional variables / Recreation of original modeling above, just isolating BTC)

## Read BTC data
df_btc = pd.read_csv('BTC-USD.csv')

## Convert 'Date' to date format
df_btc['Date'] = pd.to_datetime(df_btc['Date'])

## Quantile variable
q_model = df_quant.filter(items=[model_q], axis=0).iloc[0]['BTC-USD']

## Add 'Outcome' field 
df_btc['Outcome'] = np.where(df_btc['return_next_' + str(long)] >= q_model, 1, 0)

## Establish y and X variables
y = df_btc['Outcome']
X = df_btc.drop(columns=['Unnamed: 0', 'Outcome', 'Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
                         'return_next_1', 'return_next_' + str(short), 'return_next_' + str(med)])

## Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=seed, 
                                                    stratify=y)

## Create Logistic Regression Model
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=seed)

## Fit model on training data
classifier.fit(X_train, y_train)


## Make predictions
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

## Calculate accuracy score
accuracy = accuracy_score(y_test, predictions)

## Confusion Matrix
cm = confusion_matrix(y_test, predictions)
TP = cm[1][1]
FN = cm[1][0]
FP = cm[0][1]
TN = cm[0][0]
TP_rate = TP / (TP + FN)
TN_rate = TN / (TN + FP)
PPV_rate = TP / (TP + FP)
NPV_rate = TN / (TN + FN)

print('Confusion Matrix: \n' + str(cm) + '\n\n' + 
      'Accuracy: ' + str(round(accuracy*100,1)) + '%\n' +
      'TP_rate: ' + str(round(TP_rate*100,1)) + '%\n' +
      'TN_rate: ' + str(round(TN_rate*100,1)) + '%\n' +
      'PPV_rate: ' + str(round(PPV_rate*100,1)) + '%\n' +      
      'NPV_rate: ' + str(round(NPV_rate*100,1)) + '%\n')

#'TP_rate: ' + str(round(metrics.recall_score(y_test, predictions)*100,1)) + '%\n' + 
#'TN_rate: ' + str(round(metrics.recall_score(y_test, predictions, pos_label=0)*100,1)) + '%\n' +
#'PPV_rate: ' + str(round(metrics.precision_score(y_test, predictions)*100,1)) + '%\n' +

Confusion Matrix: 
[[323  56]
 [307  72]]

Accuracy: 52.1%
TP_rate: 19.0%
TN_rate: 85.2%
PPV_rate: 56.2%
NPV_rate: 51.3%



In [114]:
#########################################################################################################################

In [115]:
## MODEL 2 - Logistic regression - BTC (with additional variables / new start date)
## Filter date
logistic_filter = filter_date # filter_date = '2014-11-06' - first dat BTC tracked on yfinance, '2018-04-16' - first day of sentiment analysis
logistic_filter = pd.to_datetime(logistic_filter)

## Read BTC data
df_btc = pd.read_csv('BTC-USD.csv')

## Convert 'Date' to date format
df_btc['Date'] = pd.to_datetime(df_btc['Date'])

## Merge with additional variables
df_logist_model = pd.merge(df_btc, df_addl, how='left', on='Date')

## Filter dates
df_logist_model = df_logist_model.loc[df_logist_model['Date'] >= pd.to_datetime(logistic_filter)]

## Quantile variable
q_model = df_quant.filter(items=[model_q], axis=0).iloc[0]['BTC-USD']

## Add 'Outcome' field 
df_logist_model['Outcome'] = np.where(df_logist_model['return_next_' + str(long)] >= q_model, 1, 0)

## Establish y and X variables
y = df_logist_model['Outcome']
X = df_logist_model.drop(columns=['Unnamed: 0', 'Outcome', 'Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
                                  'return_next_1', 'return_next_' + str(short), 'return_next_' + str(med)])

## Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=seed, 
                                                    stratify=y)

## Create Logistic Regression Model
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=seed)

## Fit model on training data
classifier.fit(X_train, y_train)

## Make predictions
predictions = classifier.predict(X_test)
results2 = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

## Calculate accuracy score
accuracy2 = accuracy_score(y_test, predictions)

## Confusion Matrix
cm2 = confusion_matrix(y_test, predictions)
TP2 = cm2[1][1]
FN2 = cm2[1][0]
FP2 = cm2[0][1]
TN2 = cm2[0][0]
TP_rate2 = TP2 / (TP2 + FN2)
TN_rate2 = TN2 / (TN2 + FP2)
PPV_rate2 = TP2 / (TP2 + FP2)
NPV_rate2 = TN2 / (TN2 + FN2)

print('Confusion Matrix: \n' + str(cm2) + '\n\n' + 
      'Accuracy: ' + str(round(accuracy2*100,1)) + '%\n' +
      'TP_rate: ' + str(round(TP_rate2*100,1)) + '%\n' +
      'TN_rate: ' + str(round(TN_rate2*100,1)) + '%\n' +
      'PPV_rate: ' + str(round(PPV_rate2*100,1)) + '%\n' +
      'NPV_rate: ' + str(round(NPV_rate2*100,1)) + '%\n')

Confusion Matrix: 
[[323  56]
 [307  72]]

Accuracy: 52.1%
TP_rate: 19.0%
TN_rate: 85.2%
PPV_rate: 56.2%
NPV_rate: 51.3%



In [116]:
#########################################################################################################################

In [117]:
## MODEL 3 - change logistic threshold

## Set threshold
threshold = .49

## Read BTC data
df_btc = pd.read_csv('BTC-USD.csv')

## Convert 'Date' to date format
df_btc['Date'] = pd.to_datetime(df_btc['Date'])

## Quantile variable
q_model = df_quant.filter(items=[model_q], axis=0).iloc[0]['BTC-USD']

## Add 'Outcome' field 
df_btc['Outcome'] = np.where(df_btc['return_next_' + str(long)] >= q_model, 1, 0)

## Establish y and X variables
y = df_btc['Outcome']
X = df_btc.drop(columns=['Unnamed: 0', 'Outcome', 'Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
                         'return_next_1', 'return_next_' + str(short), 'return_next_' + str(med)])

## Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=seed, 
                                                    stratify=y)

## Create Logistic Regression Model
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=seed)

## Fit model on training data
classifier.fit(X_train, y_train)


## Make predictions
predictions = classifier.predict(X_test)
prediction_prob = classifier.predict_proba(X_test)
prediction_prob = pd.DataFrame(prediction_prob)[[1]]
prediction_prob
#results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
# results = pd.DataFrame({"Prediction Prob": prediction_prob, "Actual": y_test}).reset_index(drop=True)
# results
y_test = pd.DataFrame(y_test)
y_test = pd.DataFrame.reset_index(y_test)[['Outcome']]

results = pd.merge(prediction_prob, y_test, left_index=True, right_index=True)
results = results.rename(columns = {1: 'Log_Odds'})
results

## Calculate metrics
TP3 = len(results.loc[(results['Log_Odds'] >= threshold) & (results['Outcome'] == 1)])
FN3 = len(results.loc[(results['Log_Odds'] < threshold) & (results['Outcome'] == 1)])
FP3 = len(results.loc[(results['Log_Odds'] >= threshold) & (results['Outcome'] == 0)])
TN3 = len(results.loc[(results['Log_Odds'] < threshold) & (results['Outcome'] == 0)])
accuracy3 = (TP3 + TN3) / (TP3 + FN3 + FP3 + TN3)

TP_rate3 = TP3 / (TP3 + FN3)
TN_rate3 = TN3 / (TN3 + FP3)
PPV_rate3 = TP3 / (TP3 + FP3)
NPV_rate3 = TN3 / (TN3 + FN3)
cm3 = np.array([[TN3,FP3],[FN3,TP3]])

print('Confusion Matrix: \n' + str(cm3) + '\n\n' + 
      'Accuracy: ' + str(round(accuracy3*100,1)) + '%\n' +
      'TP_rate: ' + str(round(TP_rate3*100,1)) + '%\n' +
      'TN_rate: ' + str(round(TN_rate3*100,1)) + '%\n' +
      'PPV_rate: ' + str(round(PPV_rate3*100,1)) + '%\n' +
      'NPV_rate: ' + str(round(NPV_rate3*100,1)) + '%\n')

Confusion Matrix: 
[[176 203]
 [137 242]]

Accuracy: 55.1%
TP_rate: 63.9%
TN_rate: 46.4%
PPV_rate: 54.4%
NPV_rate: 56.2%



In [118]:
#########################################################################################################################

In [119]:
## Compare MODEL 1 vs MODEL 2 vs MODEL 3
df_compare = pd.DataFrame({'Metric': ['Accuracy', 'TP_Rate', 'TN_Rate', 'PPV_Rate', 'NPV_Rate'],
                        'MODEL 1': [round(accuracy*100,1), round(TP_rate*100,1), round(TN_rate*100,1), round(PPV_rate*100,1), round(NPV_rate*100,1)],
                        'MODEL 2': [round(accuracy2*100,1), round(TP_rate2*100,1), round(TN_rate2*100,1), round(PPV_rate2*100,1), round(NPV_rate2*100,1)],
                        'MODEL 3': [round(accuracy3*100,1), round(TP_rate3*100,1), round(TN_rate3*100,1), round(PPV_rate3*100,1), round(NPV_rate3*100,1)]})

df_compare.to_csv('df_compare.csv')
df_compare

,Metric,MODEL 1,MODEL 2,MODEL 3
0,Accuracy,52.1,52.1,55.1
1,TP_Rate,19.0,19.0,63.9
2,TN_Rate,85.2,85.2,46.4
3,PPV_Rate,56.2,56.2,54.4
4,NPV_Rate,51.3,51.3,56.2


In [120]:
#########################################################################################################################